# GitHub Scraping
Github의 DS_webscraping 페이지의 파일 목록을 가져와서 excel로 만들기

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup as BS
import pandas as pd

In [3]:
browser = webdriver.Chrome()
browser.get('http://github.com/login')

In [4]:
# Username 입력
browser.find_element_by_name('login').send_keys('name-chichi')

In [8]:
# Password 입력 - click
browser.find_element_by_name('password').send_keys('******')
browser.find_element_by_name('commit').click()

In [12]:
# KDT-BigData 메뉴 click
browser.find_element_by_css_selector('span[title="KDT-BigData"]').click()

In [34]:
# KDT-BigData 목록 페이지 html 가져오기
page = browser.page_source
soup = BS(page, 'html.parser')

In [33]:
# 파일/폴더명, commit message, commit date를 모두 포함하는 태그 찾아서 불러옴
tags = soup.select('.Box-row.js-navigation-item')
tags[0]

<div class="Box-row Box-row--focus-gray py-2 d-flex position-relative js-navigation-item" role="row">
<div class="mr-3 flex-shrink-0" role="gridcell" style="width: 16px;">
<svg aria-hidden="true" aria-label="Directory" class="octicon octicon-file-directory hx_color-icon-directory" data-view-component="true" height="16" version="1.1" viewbox="0 0 16 16" width="16">
<path d="M1.75 1A1.75 1.75 0 000 2.75v10.5C0 14.216.784 15 1.75 15h12.5A1.75 1.75 0 0016 13.25v-8.5A1.75 1.75 0 0014.25 3h-6.5a.25.25 0 01-.2-.1l-.9-1.2c-.33-.44-.85-.7-1.4-.7h-3.5z" fill-rule="evenodd"></path>
</svg>
</div>
<div class="flex-auto min-width-0 col-md-2 mr-3" role="rowheader">
<span class="css-truncate css-truncate-target d-block width-fit"><a class="js-navigation-open Link--primary" data-pjax="#repo-content-pjax-container" href="/BreadFeet/DS_webscraping/tree/master/files" title="files">files</a></span>
</div>
<div class="flex-auto min-width-0 d-none d-md-block col-5 mr-3" role="gridcell">
<span class="css-trun

In [20]:
# for loop으로 파일/폴더명, commit message, commit date을 각각 뽑아내기
table = []

for tag in tags:
    file = tag.select_one('div[role="rowheader"]').text.strip()
    msg = tag.select_one('.Link--secondary').text
    date = tag.select_one('time-ago').text
    table.append([file, msg, date])

table[:3]

[['files', 'Create _final file & Update .gitignore', '4 days ago'],
 ['storage', 'Create _final file & Update .gitignore', '4 days ago'],
 ['.gitignore', 'Create _final file & Update .gitignore', '4 days ago']]

In [24]:
# Dataframe으로 만들기
gh = pd.DataFrame(table, columns=['Name', 'Message', 'Date'])
gh.head()

,Name,Message,Date
0,files,Create _final file & Update .gitignore,4 days ago
1,storage,Create _final file & Update .gitignore,4 days ago
2,.gitignore,Create _final file & Update .gitignore,4 days ago
3,Day27_01_tag_object_final.ipynb,Rename files for sorting,3 hours ago
4,Day27_02_news_scraping_final.ipynb,Rename files for sorting,3 hours ago


In [30]:
# Excel 파일로 내보내기
gh.to_excel('./storage/github.xlsx', header=True, index=False)
print('Dataframe saved to excel-----------------------------')

Dataframe saved to excel-----------------------------


In [32]:
# Excel에서 dataframe 불러오기
pd.read_excel('./storage/github.xlsx', header=0).head()

,Name,Message,Date
0,files,Create _final file & Update .gitignore,4 days ago
1,storage,Create _final file & Update .gitignore,4 days ago
2,.gitignore,Create _final file & Update .gitignore,4 days ago
3,Day27_01_tag_object_final.ipynb,Rename files for sorting,3 hours ago
4,Day27_02_news_scraping_final.ipynb,Rename files for sorting,3 hours ago


---
# Incruit Website Scraping
incruit.com 페이지에서 데이터 분석가 검색하여서 채용정보 스크래핑하기

In [35]:
browser = webdriver.Chrome()
browser.get('https://www.incruit.com')

In [49]:
# 검색창에 데이터 분석 입력 
from selenium.webdriver.common.keys import Keys
elmt = browser.find_element_by_name('kw')
elmt.send_keys('데이터 분석가')

아래 사용되는 Keys의 값을 확인하고 싶으면: https://www.selenium.dev/selenium/docs/api/py/webdriver/selenium.webdriver.common.keys.html 

In [50]:
# Enter 쳐서 창 넘어가기
from selenium.webdriver.common.keys import Keys
elmt.send_keys('\ue007')            # Keys.Enter = '\ue007' 어느 것을 입력해도 된다

In [53]:
# '채용' 메뉴 클릭
browser.find_element_by_link_text('채용').click()

이후 크롤링 방법은 Selenium만 이용해서 해본다.

In [134]:
# Pagenation으로 3페이지까지 가져오기
# Explicit Wait 사용하기 위해서 모듈 import
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'li > h3 > a')))

incruit = []

for i in range(1, 4):
    if i != 1:                # 1페이지는 클릭할 필요 없음
        browser.find_element_by_link_text(str(i)).click()
    
    # 하나의 목록 내용을 모두 가져오는 태그 찾기
    tags = browser.find_elements_by_css_selector('ul.litype01 > li')
   
    # for loop으로 회사명, 채용내용, 정규직여부, 위치, 지원요건, 경력, 연봉 모두 추출
    for tag in tags:    
        company = tag.find_element_by_css_selector('li > h3').text.replace('관심기업등록', '')  
        title = tag.find_element_by_css_selector('span > a').text
        txt = tag.find_element_by_css_selector('p.etc > span').text     # 정규직여부~연봉까지 내용은 하나의 text로 묶여있음!
        status = txt.split('|')[0].strip()
        loc = txt.split('|')[1].strip()
        req = txt.split('|')[2].strip()
        exp = txt.split('|')[3].strip()
        sal = txt.split('|')[4]
        incruit.append([company, title, status, loc, req, exp, sal])

In [135]:
incruit[:4]

[['(주)국민은행',
  'KB국민은행 비정형 데이터 분석가 전문직무직원 채용',
  '계약직',
  '서울>마포구',
  '학력무관',
  '경력1년이상',
  ' 연봉 협의 후 결정'],
 ['(주)데이터누리',
  'PM/PL & 데이터분석가 & 일반영업 정규직 채용',
  '정규직',
  '서울>강서구',
  '대학(2,3년)졸업',
  '경력2년이상',
  ' 연봉 회사내규'],
 ['리디주식회사',
  '[리디 주식회사] 콘텐츠 데이터분석가를 모집합니다',
  '정규직',
  '서울>강남구',
  '학력무관',
  '경력3~7년',
  ' 연봉 협의 후 결정'],
 ['어니컴(주)',
  '[어니컴(주)] 데이터 분석가 신입/경력 모집',
  '정규직',
  '서울>용산구',
  '대학교(4년)졸업이상',
  '신입/경력(연차무관)',
  ' 연봉 회사내규']]

In [129]:
tags[0].get_attribute('outerHTML')           # tag 구조 확인

'<li><h3><a href="https://www.incruit.com/company/1682268209" target="_blank">환경분석연구원(주)</a><button type="button" class="add_company" f_scrapcompany="" f_comp="4DC58DB03D69E3BDD470F0D269B05A77702709758C927E8F77ED1F3E0D08623A" f_mem="3B99353088F33084B709CED6C20917C40275B624F2A533E5456F7485B1EFB383" f_act="CI">관심기업등록</button></h3><p class="detail"><span class="rcrtTitle"><a href="https://job.incruit.com/jobdb_info/jobpost.asp?job=2106080001309&amp;src=gsw*search" slot-sort="19463556-0" target="_blank">실내공기질 측정팀으로 모집합니다(관련학과 필수)</a></span><span class="info">(~07/08)</span><span class="dday">D-9</span><button type="button" class="add_scrap" f_job="2106080001309" f_act="I" f_type="INCJOB">스크랩</button><button type="button" bl_job="2106080001309" class="apply_job" onclick="goCL(this,\'19044\',\'search\',\'목록_바로지원_2106080001309\');">바로지원</button></p><p class="etc"><span>정규직 | 경기&gt;성남시 중원구 | 대학(2,3년)졸업이상 | 신입/경력(연차무관) | 연봉 3,000~3,500만원</span><span><a href="#" onclick="open_win(\'https://job.i

In [130]:
# Dataframe으로 만들기
incr = pd.DataFrame(incruit, columns=['Company', 'Title', 'Status', 'Location', 'Requirement', 'Experience', 'Salary'])
incr.head()

,Company,Title,Status,Location,Requirement,Experience,Salary
0,(주)국민은행,KB국민은행 비정형 데이터 분석가 전문직무직원 채용,계약직,서울>마포구,학력무관,경력1년이상,연봉 협의 후 결정
1,(주)데이터누리,PM/PL & 데이터분석가 & 일반영업 정규직 채용,정규직,서울>강서구,"대학(2,3년)졸업",경력2년이상,연봉 회사내규
2,리디주식회사,[리디 주식회사] 콘텐츠 데이터분석가를 모집합니다,정규직,서울>강남구,학력무관,경력3~7년,연봉 협의 후 결정
3,어니컴(주),[어니컴(주)] 데이터 분석가 신입/경력 모집,정규직,서울>용산구,대학교(4년)졸업이상,신입/경력(연차무관),연봉 회사내규
4,주식회사 토이스미스,경력직 데이터분석가 채용,정규직,서울>강남구,학력무관,경력2년이상,연봉 협의 후 결정


In [131]:
# Excel로 만들기
incr.to_excel('./files/incruit.xlsx', header=True, index=False)
print('Converted to Excel file---------------------------------')

Converted to Excel file---------------------------------
